# I. Issues

While writing the code for this project, I ran into two big issues:
1. There are no predictive variables, so it is not possible to build a predictive model. The only analysis that can be done is classification of past behavior by doctors, so our prediction of the future here will be the average of the past. This prediction can only be done where there is relevant data, so it won't work for most doctors as the vast majority of doctors dealing in gastroenterology have not had a patient with only a benign growth in the intestine (see Appendix I).
2. Crucial information about the database (whether patients were referred or walked into the ER, which doctor is represented by the doctor_id, the time frame over which the data was taken) was absent. In this specific case, this  missing information makes the desired analysis very likely to be wrong. Since there is only one doctor ID, we do not know what that ID represents - the doctor who made the referral, the doctor who is responsible for the patient, or the head doctor performing the surgeries. There are so many edge cases that knowing what the doctor_id represents is absolutely necessary to a functional and truthful analysis. 

As I understood it, the rule for this case is as follows: 
*If a patient has a colonoscopy which does not successfully remove a growth, they should be referred to a different place for another colonoscopy - they should NOT be referred to a colectomy. Find how often doctors refer patients in said situation to a colectomy.*

In [2]:
import pandas as pd

from ExtractData import Codes, Query
from CleanseData import Cleanse
from AnalyzeData import Analyze

# II. Data Hunting and Gathering
The first thing I like to do when starting a new project is to spend some time thoroughly understanding the data. Here, we have two datasets in claims.db: medical_headers and medical_service_lines. In my code, I call medical_headers "visits" since it contains ICD-9 codes for the diagnoses and procedures performed during the patient's visit to the hospital (which shall henceforth be referred to as the "encounter"), other data like the diagnosis group the patient was assigned, the total charge for the visit, and codes representing the patient's status upon arrival and discharge, as well as ids for the encounter, the patient, the doctor, and the hospital. I call medical_service_lines "services" since it contains CPT codes for procedures performed (these are different codes than the ICD-9 codes, although sometimes different codes for the same type of procedure will show up in both datasets) and encounter keys which match the medical_headers dataset.

I wasn't provided with any information describing what each of the columns represented, so I figured that was something I had to figure out on my own. My own research proved satisfactory, except for a few questions I was unable to answer on my own:
- Who, *specifically*, does the doctor_id column represent?
- The majority of arrival codes correspond to Emergency, Urgent, or Elective. For which of these could a patient have arrived with a referral? I think Emergency and Urgent are without referrals and Elective is with a referral, but I'm not sure.
- What do the 'modifier' and 'units' columns represent in the larger dataset?

The instructions file specifies that solution code should "[run] without errors or user input" but also "[have] the ability to handle a slightly different rule set." There's a bit of a contradiction here, since to have a different rule set you must choose a different set of codes, which requires user input. I create a class Codes which asks the user which medical codes to input (including ICD-9 codes ending in a star and a range of CPT codes), and stores those codes as attributes. I then create a class Query which takes as an argument lists of ICD-9 and CPT codes, which can be retrieved from the Codes class or input manually as a list. The Query class then constructs a SQL query to grab all data containing those codes from the relevant databases and returns that as two DataFrames corresponding to the data taken from medical_headers and medical_service_lines.

For the Codes and Query classes, the ICD-9 and PCT codes are stored as attributes, and the Query class stores the full SQL commands as attributes as well.

In [3]:
dummy = Codes()

Please input all Diagnosis ICD-9-CM codes, one at a time.              

If you wish to select all codes after a decimal, place a * after the final digit.              
For example, if you wished to include all codes beginning with 271, you would enter 271.*              
If you wished to include all codes beginning with 271.8, you would input 271.8*              

Once you are finished entering codes, enter "done".              

If you would like to use the default codes of 211.3, 211.4 for benign polyps              
and 152.* for malignant growths, enter "default".
default

 CODES: Your Diagnosis ICD-9-CM codes are ['152.*', '211.3', '211.4']


Please input all Procedure ICD-9-PCS codes, one at a time.              

If you wish to select all codes after a decimal, place a * after the final digit.              
For example, if you wished to include all codes beginning with 271, you would enter 271.*              
If you wished to include all codes beginning with 271.8, you would in

In [3]:
q = Query(dummy.D_codes, dummy.P_codes, dummy.CPT_codes)

In [5]:
visits, services = q.get_dataframes()
print('Shapes of DataFrames:', visits.shape, services.shape)
visits.head()

Shapes of DataFrames: (382937, 53) (331658, 8)


,index,encounter_key,claim_type_code,icd_type,discharge_status_code,admit_type_code,DA,D1,D2,D3,...,P11,P12,P13,P14,P15,patient_id,total_claim_charge_amount,bill_type_code,doctor_id,hospital_id
0,7,ae7cf8585c1bda2be18091c821eee5846c29737adc7255...,I,9,,,312,312,152.1,,...,,,,,,d73675f4ced179dd89841023520add308ca9506c3b7b68...,000000042104,131,f4a0d6867e756d3a58a040e1eb4bbe4445f55eb7bc6965...,2c56dde98e747996a6f26fc1eb39d00e4d4730048b6d2d...
1,33,6d33ed96e3e9b694ef98cc6c02965d239a0c61c6f7891c...,I,9,,,,309.28,152.0,,...,,,,,,ae9c79dbb1705869914e49b4672a526f67203608fe4eb4...,000000017600,131,7da9a88582cb5e57f7b60680e1d576e9c18d7b9d6a0e33...,60e1212ea346becd0bc3caf0e74f70af9c1c36a740855e...
2,35,800150c2f4cfc2b1352eaeee8c22e014734572725054ed...,I,9,,,,,152.2,27.93,...,,,,,,09948392c93d3cc562836d8cb835d181b2b6cb26c42305...,000000105900,131,16f393b27dc81a6792e0d3ec41789501b68a1808eac0a5...,d318cc6952020bb372f79a03ada0c22ea52d979e66c06a...
3,36,273d4f32d7f0643a4223901c2eb4690960f24fd8fcd4e7...,I,9,,,,312.34,152.1,,...,,,,,,909858918678003375d75b44ecd2bd4eae1450513b0852...,000000017600,131,a04894e573bb36afbaeca27442592c846a45d058787cbf...,60e1212ea346becd0bc3caf0e74f70af9c1c36a740855e...
4,38,1720053add288946cb22bc431bafbeae02ab21a8d43f49...,I,9,,,,9.7,152.2,,...,,,,,,70f54de4003d6c4004530eb40b2c8f31a60c18c5c2c2dc...,000000155687,131,64f834baa2508046e92544c4263c5323058438db5e12e7...,3888a29e4368a50f62c568e904c00a4b97f0c6fd73a842...


In [6]:
services.head()

,index,encounter_key,procedure,modifier1,modifier2,units,line_charge,revenue_code
0,12,0bbaee1691a952f0e4525c01f688bb7969d4770d1c677c...,44110,None,None,None,None,None
1,22,8d693e9405e0908fa9027c619c44943dea3decddb5de0e...,44152,None,None,00000000,0000008900,0430
2,33,789ef32225bfae753058d7c78c20ce570adc808048e906...,44152,None,None,00000000,0000636287,0480
3,64,5fdd1a8babd792df500acb61bbab0be8d41eabd378a348...,44151,None,None,None,None,None
4,98,3f4ebe7720a7a3235b8389fb150a51d9ce5413e98a024a...,44151,None,None,None,None,None


# III. Wrangle the Data
For wrangling the data, the first thing I did was a left merge of medical_headers (which I call visits) and medical_service_lines (which I call services), so that services was merged into visits. I then drop the unnecessary index columns as well as the columns with a cardinality of 1, which just inform us that our codes are ICD-9 codes. The empty cells in visits are left as blank strings, while the empty cells in services are given a None value, so I replace all of them with NaN values.

Many of the columns should be numeric, so I change them to floats. Then I feature engineer four new binary columns called 'benign', 'malignant', 'colonoscopy', and 'colectomy' to denote if a benign and/or malignant growth was detected, and if a colonoscopy and/or colectomy was performed for that encounter. I save this dataframe as "df", since I want to have one DataFrame with every piece of information, as it become relevant in a later analysis.

I clean the data further by dropping all columns that have no relevance to the analysis I want to do, with the exception of the total cost of the encounter, which I keep in the hopes that it might be useful in calculating the cost of mistaken surgeries. I reorder the remaining columns, placing the encounter key first. I *do not* make it the index, as there are duplicate values for some encounters for reasons I will describe shortly. I save this DataFrame as data_full, in case it's necessary to have a version of the data with the original strings.

I notice that all the ids are long strings. I decide I want them to be numeric, so I can look for interesting patterns. I separate my data into two new DataFrames: data, which contains everything relevant to the analysis, and ids, which matches all the old string ids to the new numeric ids.

One very important thing that I discovered was that in medical_service_lines there are sometimes multiple procedures in multiple rows for the same encounter. This proves to be a problem when merging the datasets together, as the resulting dataset will have duplicate rows for the same encounter, each describing a different CPT procedure. I address this by merging the duplicate rows and performing an OR operation on each of the binary features I engineered to ensure no information is lost. I considered if I should make the data long instead, but I think this approach is superior.

In [7]:
c = Cleanse()
data, ids, data_full, df = c.wrangle(visits, services)

In [8]:
data.head()

,new_encounter_key,benign,malignant,colonoscopy,colectomy,total_claim,new_patient_id,new_doctor_id,new_hospital_id
0,0,0,1,0,0,42104.0,0,0,0
1,1,0,1,0,0,17600.0,1,1,1
2,2,0,1,1,1,105900.0,2,2,2
3,3,0,1,0,0,17600.0,3,3,1
4,4,0,1,0,0,155687.0,4,4,3


# IV. Analyzing If A Doctor Broke The Rule
For this Analysis, I assumed that the doctor_id connected to an encounter represented the doctor responsible for the referral. I quickly realized that this assumption was probably wrong, but even after several days of thinking was unable to come up with a suitable replacement. To be clear, the Analysis is done correctly, but I'm pretty sure my assumption is faulty, making my Analysis faulty as well. 

In my Analyze class, it takes the data and ids DataFrames from the Cleanse class as inputs, and the method get_mistakes() returns a DataFrame of each doctor who had at least 1 patient with a benign tumor and not a malignant tumor, ranked by how often those patients had surgery (the percentage rate at which the doctor made mistakes) and sub-ranked by how many chances the doctor had to make those mistakes. I include both the numeric doctor_id I created and the original string.

From the assignment, the 'chances' column is my denominator, the 'mistakes' column is my numerator, and the '% mistakes' column is my fraction.

In [9]:
a = Analyze(data, ids)
mistakes = a.get_mistakes()
mistakes

,new_doctor_id,chances,mistakes,% mistakes,doctor_id
0,9109,17,0,0.0,155c9dfd9684eee2166818ae026b6ae26349beaa8a078c...
1,9418,15,0,0.0,5151a4d921dd6779d82a1e199fd6fd9bec16aaece90d24...
2,2392,14,0,0.0,60ef03fb2cbf699d532ace2f9a04ab7ce3db0200ae0e9a...
3,5579,14,0,0.0,b10f655694b0e925e60172d2add190d4bc0c50b83cd246...
4,9323,14,0,0.0,d0302952184e894de108d722d3b046ed24e60f647ebd2c...
...,...,...,...,...,...
1894,33493,1,1,100.0,8ae96d67fd375cd21965c5f86a2e93c812295523b8c685...
1895,33549,1,1,100.0,63c2cd4515271b70060fa2713fcf9f0bfa3872c3a3f4ae...
1896,22783,1,1,100.0,f0e6fe55eab7fdd7486c8156b72032747a447525caff84...
1897,14257,1,1,100.0,9257260e9d3516be98cda8cdd6a7acf4982b0c778462ec...


# V. Articulating Axioms and Assumptions - Is The Analysis Correct?
While analyzing the data, I became increasingly struck by how important choosing my axioms and assumptions were in extracting, cleaning, and analyzing the data. Here I lay out what I accepted as axioms, what I chose as assumptions, and what I discovered to be very pressing questions which dramatically affect the eff
## Axioms
- The ICD-9 and PCT codes were entered correctly and fully.
- The psedonymous ids for encounters, patients, doctors, and hospitals are consistent across the data.

## Assumptions
- The encounter keys are ordered in time. My justification for this is that, for patients who visit a hospital multiple times, there are often groupings of encounters. I believe this represents how sometimes when people get sick they will have to visit the hospital multiple times in a short time frame or stay in a hospital room for an extended period.
- If a surgery occurred, the doctor_id in the same encounter as the surgery is the doctor who gave the referral for the surgery. I must make this assumption to analyze which doctors are referring patients with beingn polpys for a colectomy, otherwise I have nothing to analyze. I think this assumption is wrong, and so my analysis is probably wrong.
-- *IF* the doctor_id represents the surgeon, or any doctor besides the referring doctor, I have no information as to the id representing the referring doctor. If this is true, the only possibility that this dataset can be used for this analysis is that the previous doctor the patient saw was the referring doctor - but from what I understand, the doctors who refer patients to hospitals for surgery are not usually a part of the hospital system, and thus the encounter from which the patient received a referral may not be in the database.
- The doctor_id represents a single doctor. The desired output from the assignment is a ranking of doctors based on how often they sent patients with only a benign tumor to surgery - it seems implied that single doctors are the target of the analysis. However, there is one WEIRD doctor in the dataset who accounts for an astonishing 16,570 encounters, which is 4.3% of the relevant data. See Appendix I for more info. This cannot possibly represent a single doctor, as the next most prolific doctor_id appears only 939 times. Relevant here is the time span of the data, which is unknown. 

## Questions
- What is an encounter - is it a day at the hospital or something else? What does it mean when a patient has encounters that are very close together? The closeness can be different, as the encounter_key can jump by just one or several hundred. My best guess is each encounter represents a day in the hospital, although that does not explain why, in groups of encounters for the same patient, some encounters are grouped more closely than others. 
- Who does the doctor_id represent? A patient undergoing diagnoses and procedures may see several different doctors, i.e. a doctor that checks up on them, a doctor that performs surgery, and a doctor that gave them a referral to the surgery, if not others. Also, why does one doctor_id account for 4.3% of the encounters? 
- Patients who have surgeries performed at a hospital can have arrived there after having received a referral from a doctor, *OR* they can also be diagnosed by a walk-in visit to the ER. I can use the admit_type_code to distinguish this, but I'm not sure how to determine if a referral occurred. 
- How many patients with a benign polyp receive a referral for surgery but don't go? Can I tell when this has occurred?
- How can I tell if a colonoscopy was unsuccessful? I think that it's usually the colonoscopy itself which determines if a growth is benign or malignant. So if a colonoscopy fails to remove a growth, we can't know if it's benign or malignant. So should the analysis instead be rephrased to "if a patient's colonoscopy fails to remove a growth and the patient does not have a malignant growth, find the rate at which those patients are sent to surgery"?
- Is the medical journal paper found in the Garner folder relevant? I'm pretty sure the only information I have available is if a benign and/or malignant growth was present, not how many of them were observed. 

### The two most important questions, which the entire endeavor rests upon, are:
- *How can I tell if a referral for surgery has occurred?*
- *How can I know who made the referral?*

I do not know how to answer these questions, nor do I know where to look on the internet to answer these questions. I think these questions would require specific knowledge of hospitals that I do not have, and/or asking someone connected to the dataset itself.

In [10]:
# Appendix I

# show that one weird doctor accounts for 16,570 encounters and 4.3% of the overall encounters
weird_doctor = data.loc[data['new_doctor_id'] == 234]
print(f'Weird doctor was in {len(weird_doctor)} encounters, \
{len(weird_doctor)*100 / len(data)}% of overall encounters')

# show only 5.6% of doctors in gastroenterology have had a patient with only a benign tumor
benign_data = data.loc[(data['benign'] == 1) & (data['malignant'] == 0)]
print('Percent of doctors who had a patient with only a benign growth:', 
      len(benign_data['new_doctor_id'].unique()) / len(data['new_doctor_id'].unique())*100)


Weird doctor was in 16570 encounters, 4.3270825227126135% of overall encounters
Percent of doctors who had a patient with only a benign growth: 5.653468294135159


# To-Do
- Rewrite the invalid check in Codes() into a function to check if the ICD-9 codes are floats between 0 and 1000 and if the CPT codes are 5-digit integers. Reject the codes otherwise.
- Modify Query() so that the code runs correctly if there are no CPT codes or no ICD-9 codes. Maybe raise an error? Cleanse() will require both visits and services DataFrames.
- Maybe update Cleanse() so that the visits and services DataFrames can be passed as arguments? I might try class inheritance, too.
- Feature engineer three columns denoting the number of ICD-9 diagnoses, ICD-9 procedures, and PCT procedures.
- Feature engineer three columns which contain lists of strings for the codes for ICD-9 diagnoses, ICD-9 procedures, and PCT procedures.
- Create a general class in CleaseData.py for a new medical rule, call it GeneralCleanse()? Need to know what sort of structure other medical rules follow before I can make this...